In [29]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools

# Loading dataset's

In [4]:
credits_ds = pd.read_csv('./dataset/IMDB/credits.csv')
credits_ds

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [5]:
links_ds = pd.read_csv('./dataset/IMDB/links_small.csv')
links_ds

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [6]:
ratings_ds = pd.read_csv('./dataset/IMDB/ratings_small.csv')
ratings_ds

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [7]:
keywords_ds = pd.read_csv('./dataset/IMDB/keywords.csv')
keywords_ds

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [8]:
movies_metadata_ds = pd.read_csv('./dataset/IMDB/movies_metadata.csv') 
movies_metadata_ds

C:\Users\parsa\AppData\Local\Temp\ipykernel_9420\4219932325.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_ds = pd.read_csv('./dataset/IMDB/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [9]:
movies_metadata_ds = movies_metadata_ds[movies_metadata_ds['status'] == 'Released']
movies_metadata_ds = movies_metadata_ds[movies_metadata_ds['vote_count'] > 40]
movies_metadata_ds = movies_metadata_ds[movies_metadata_ds['vote_average'] >= 5]
important_col = ['id', 'genres', 'overview', 'original_title', 'belongs_to_collection']
movies_metadata_ds = movies_metadata_ds[important_col]
movies_metadata_ds.reset_index(inplace=True, drop=True)
movies_metadata_ds['genres'] = movies_metadata_ds['genres'].apply(lambda x: ' '.join([i['name'].lower().replace(' ', '') for i in eval(x)]))
movies_metadata_ds['belongs_to_collection'] = movies_metadata_ds['belongs_to_collection'].apply(lambda x: eval(str(x))['name'].lower().replace(' ', '') if str(x).lower() != 'nan' else '')

movies_metadata_ds

,id,genres,overview,original_title,belongs_to_collection
0,862,animation comedy family,"Led by Woody, Andy's toys live happily in his ...",Toy Story,toystorycollection
1,8844,adventure fantasy family,When siblings Judy and Peter discover an encha...,Jumanji,
2,15602,romance comedy,A family wedding reignites the ancient feud be...,Grumpier Old Men,grumpyoldmencollection
3,11862,comedy,Just when George Banks has recovered from his ...,Father of the Bride Part II,fatherofthebridecollection
4,949,action crime drama thriller,"Obsessive master thief, Neil McCauley leads a ...",Heat,
...,...,...,...,...,...
9613,265189,comedy drama,"While holidaying in the French Alps, a Swedish...",Turist,
9614,277839,comedy,Franky and Krimo dream of leaving the grey gri...,Pattaya,
9615,430365,comedy,Jean-Étienne Fougerole is an intellectual bohe...,À bras ouverts,
9616,455661,family animation romance comedy,A closeted boy runs the risk of being outed by...,In a Heartbeat,


In [10]:
credits_ds['cast'] = credits_ds['cast'].apply(lambda x: ' '.join([i['name'].lower().replace(' ', '') for i in eval(x)]))

credits_ds['crew'] = credits_ds['crew'].apply(lambda x: [i['name'].lower().replace(' ', '') if i['job'] == 'Director' else '' for i in eval(x)])
credits_ds['crew'] = credits_ds['crew'].apply(lambda x: ' '.join([i for i in x if i != '']))
credits_ds['cast'] = credits_ds.apply(lambda x: x.loc['cast'] + ' ' + x.loc['crew'], axis=1)
credits_ds = credits_ds[['id', 'cast']]
credits_ds.reset_index(inplace=True, drop=True)
credits_ds

,id,cast
0,862,tomhanks timallen donrickles jimvarney wallace...
1,8844,robinwilliams jonathanhyde kirstendunst bradle...
2,15602,waltermatthau jacklemmon ann-margret sophialor...
3,31357,whitneyhouston angelabassett lorettadevine lel...
4,11862,stevemartin dianekeaton martinshort kimberlywi...
...,...,...
45471,439050,leilahatami kouroshtahami elhamkorda hamidnema...
45472,111109,angelaquino perrydizon hazelorencio joeltorre ...
45473,67758,erikaeleniak adambaldwin juliedupage jamesrema...
45474,227506,iwanmosschuchin nathalielissenko pavelpavlov a...


In [11]:
keywords_ds['keywords'] = keywords_ds['keywords'].apply(lambda x: ' '.join([i['name'].lower().replace(' ', '') for i in eval(x)]))
keywords_ds

,id,keywords
0,862,jealousy toy boy friendship friends rivalry bo...
1,8844,boardgame disappearance basedonchildren'sbook ...
2,15602,fishing bestfriend duringcreditsstinger oldmen
3,31357,basedonnovel interracialrelationship singlemot...
4,11862,baby midlifecrisis confidence aging daughter m...
...,...,...
46414,439050,tragiclove
46415,111109,artist play pinoy
46416,67758,
46417,227506,


In [12]:
movies_metadata_ds = movies_metadata_ds[movies_metadata_ds['id'].str.isnumeric()]
movies_metadata_ds['id'] = movies_metadata_ds['id'].astype(int)

df = pd.merge(movies_metadata_ds, keywords_ds, on='id', how='left')
df = pd.merge(df, credits_ds, on='id', how='left')
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df

,id,genres,overview,original_title,belongs_to_collection,keywords,cast
0,862,animation comedy family,"Led by Woody, Andy's toys live happily in his ...",Toy Story,toystorycollection,jealousy toy boy friendship friends rivalry bo...,tomhanks timallen donrickles jimvarney wallace...
1,8844,adventure fantasy family,When siblings Judy and Peter discover an encha...,Jumanji,,boardgame disappearance basedonchildren'sbook ...,robinwilliams jonathanhyde kirstendunst bradle...
2,15602,romance comedy,A family wedding reignites the ancient feud be...,Grumpier Old Men,grumpyoldmencollection,fishing bestfriend duringcreditsstinger oldmen,waltermatthau jacklemmon ann-margret sophialor...
3,11862,comedy,Just when George Banks has recovered from his ...,Father of the Bride Part II,fatherofthebridecollection,baby midlifecrisis confidence aging daughter m...,stevemartin dianekeaton martinshort kimberlywi...
4,949,action crime drama thriller,"Obsessive master thief, Neil McCauley leads a ...",Heat,,robbery detective bank obsession chase shootin...,alpacino robertdeniro valkilmer jonvoight toms...
...,...,...,...,...,...,...,...
9752,265189,comedy drama,"While holidaying in the French Alps, a Swedish...",Turist,,femalenudity darkcomedy familyvacation avalanc...,lisalovenkongsli johannesbahkuhnke clarawetter...
9753,277839,comedy,Franky and Krimo dream of leaving the grey gri...,Pattaya,,,ramzybedia malikbentalha franckgastambide gade...
9754,430365,comedy,Jean-Étienne Fougerole is an intellectual bohe...,À bras ouverts,,,christianclavier aryabittan elsazylberstein cy...
9755,455661,family animation romance comedy,A closeted boy runs the risk of being outed by...,In a Heartbeat,,love teenager lgbt short,bethdavid estebanbravo


In [13]:
col = list(df.columns)
col.remove('id')
col.remove('genres')
col.remove('original_title')
df['title'] = df['original_title']
df['token'] = df['genres']
for i in col:
    df ['token'] = df['token'] + ' ' + df[i]
df = df[['id', 'title', 'token']]
df

,id,title,token
0,862,Toy Story,"animation comedy family Led by Woody, Andy's t..."
1,8844,Jumanji,adventure fantasy family When siblings Judy an...
2,15602,Grumpier Old Men,romance comedy A family wedding reignites the ...
3,11862,Father of the Bride Part II,comedy Just when George Banks has recovered fr...
4,949,Heat,action crime drama thriller Obsessive master t...
...,...,...,...
9752,265189,Turist,comedy drama While holidaying in the French Al...
9753,277839,Pattaya,comedy Franky and Krimo dream of leaving the g...
9754,430365,À bras ouverts,comedy Jean-Étienne Fougerole is an intellectu...
9755,455661,In a Heartbeat,family animation romance comedy A closeted boy...


In [14]:
df.drop(df[df['token'].isnull()].index, inplace=True)
df

C:\Users\parsa\AppData\Local\Temp\ipykernel_9420\1594095297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['token'].isnull()].index, inplace=True)


,id,title,token
0,862,Toy Story,"animation comedy family Led by Woody, Andy's t..."
1,8844,Jumanji,adventure fantasy family When siblings Judy an...
2,15602,Grumpier Old Men,romance comedy A family wedding reignites the ...
3,11862,Father of the Bride Part II,comedy Just when George Banks has recovered fr...
4,949,Heat,action crime drama thriller Obsessive master t...
...,...,...,...
9752,265189,Turist,comedy drama While holidaying in the French Al...
9753,277839,Pattaya,comedy Franky and Krimo dream of leaving the g...
9754,430365,À bras ouverts,comedy Jean-Étienne Fougerole is an intellectu...
9755,455661,In a Heartbeat,family animation romance comedy A closeted boy...


In [15]:
tfidf = TfidfVectorizer(max_features=5000)
vectorized_data = tfidf.fit_transform(df['token'].values)
vectorized_data

<9716x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 413552 stored elements in Compressed Sparse Row format>

In [16]:
similarity = cosine_similarity(vectorized_data)
similarity

array([[1.        , 0.04011667, 0.02115663, ..., 0.04573757, 0.11700804,
        0.0376136 ],
       [0.04011667, 1.        , 0.07140257, ..., 0.08394441, 0.01994877,
        0.04027354],
       [0.02115663, 0.07140257, 1.        , ..., 0.0780224 , 0.03333599,
        0.10092987],
       ...,
       [0.04573757, 0.08394441, 0.0780224 , ..., 1.        , 0.13819079,
        0.06540503],
       [0.11700804, 0.01994877, 0.03333599, ..., 0.13819079, 1.        ,
        0.07908798],
       [0.0376136 , 0.04027354, 0.10092987, ..., 0.06540503, 0.07908798,
        1.        ]])

In [17]:
def recommend_by_movie(title, number=20):
    if len(df[df['title'] == title]) == 0:
        return []
    movie_id = df[df['title'] == title].index[0]
    distances = similarity[movie_id]
    movies = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])
    return [df.iloc[i[0]].title for i in movies[:number]]

print(recommend_by_movie('Toy Story', 10))

print('***************************************************************************')

print(recommend_by_movie('Jumanji', 10))

print('***************************************************************************')

print(recommend_by_movie('Rocky III', 10))

['Toy Story', 'Toy Story 2', 'Toy Story 3', 'The 40 Year Old Virgin', 'Hawaiian Vacation', 'Partysaurus Rex', 'Small Fry', 'Toy Story of Terror!', 'Toy Story That Time Forgot', '桃姐']
***************************************************************************
['Jumanji', 'Brainscan', 'Stay Alive', 'Wreck-It Ralph', 'Nirvana', 'Nerve', '12 Rounds 2: Reloaded', 'Panic Room', 'The Last Starfighter', 'Knights of Badassdom']
***************************************************************************
['Rocky III', 'Rocky IV', 'Rocky V', 'Rocky Balboa', 'Creed', 'It Could Happen to You', 'Rocky II', 'Rocky', 'The Ghost Writer', 'Cinderella Man']


In [30]:
def recommen_by_user(user_id, number=20):
    user_rate_ds = ratings_ds[ratings_ds['userId'] == user_id]
    sort = user_rate_ds.sort_values(by='rating', ascending=False)
    movie_id = sort['movieId']
    movie_list = [df[df['id'] == id]['title'].values[0] for id in movie_id if len(df[df['id'] == id]['title']) > 0]
    result = [recommend_by_movie(str(title)) for title in movie_list]
    return list(itertools.chain.from_iterable(result))

recommen_by_user(1)

['American Pie',
 'American Reunion',
 'Date and Switch',
 'High School Musical 3: Senior Year',
 'American Pie 2',
 'American Pie Presents: The Book of Love',
 'Premature',
 'Another Gay Movie',
 'Very Good Girls',
 'Immaturi',
 'Sleepover',
 'The Breakfast Club',
 'Good Kids',
 "Romy and Michele's High School Reunion",
 'Grease',
 "Bill & Ted's Excellent Adventure",
 'Get a Job',
 "Porky's",
 "Comment c'est Loin",
 'Boys and Girls',
 'Rocky III',
 'Rocky IV',
 'Rocky V',
 'Rocky Balboa',
 'Creed',
 'It Could Happen to You',
 'Rocky II',
 'Rocky',
 'The Ghost Writer',
 'Cinderella Man',
 'Ant-Man',
 'Bleed for This',
 'Never Back Down: No Surrender',
 'Елена',
 'The Fighter',
 'Kickboxer',
 'Eye of the Beholder',
 'Beyond a Reasonable Doubt',
 'Até que a Sorte nos Separe',
 'ブラム',
 'Jay and Silent Bob Strike Back',
 'Chasing Amy',
 'Dogma',
 'Clerks II',
 'Mallrats',
 'Clerks',
 'Jersey Girl',
 'Scream 3',
 'Maps to the Stars',
 'Electric Boogaloo: The Wild, Untold Story of Cannon Fil

# Colaborative

In [19]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
movie_df = pd.read_csv('./dataset/IMDB/movies_metadata.csv')
movie_df

C:\Users\parsa\AppData\Local\Temp\ipykernel_9420\514200638.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_df = pd.read_csv('./dataset/IMDB/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [21]:
rate_df = pd.read_csv('./dataset/IMDB/ratings_small.csv')
rate_df

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [22]:
user_max = rate_df['userId'].max()
movie_max = rate_df['movieId'].max()

i = torch.LongTensor(rate_df[['userId', 'movieId']].to_numpy())
v = torch.FloatTensor(rate_df[['rating']].to_numpy().flatten())

sparse_matrix = torch.sparse.FloatTensor(i.t(), v, torch.Size([user_max+1, movie_max+1])).to_dense()

print(sparse_matrix.shape)
sparse_matrix

torch.Size([672, 163950])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 4., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [23]:
similarities_sparse = cosine_similarity(sparse_matrix, dense_output=False)
print(similarities_sparse.shape)
similarities_sparse

(672, 672)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.0000001 , 0.        , ..., 0.06291708, 0.        ,
        0.01746565],
       [0.        , 0.        , 1.        , ..., 0.02413984, 0.17059463,
        0.1131753 ],
       ...,
       [0.        , 0.06291708, 0.02413984, ..., 1.0000001 , 0.04260878,
        0.08520195],
       [0.        , 0.        , 0.17059463, ..., 0.04260878, 0.99999994,
        0.22867674],
       [0.        , 0.01746565, 0.1131753 , ..., 0.08520195, 0.22867674,
        1.        ]], dtype=float32)

In [24]:
def top_n_idx_sparse(user_id, number=20):
    user_row = similarities_sparse[user_id]
    user_details = list(map(lambda x: (x[0], x[1]), enumerate(user_row)))
    sort = list(sorted(user_details, key=lambda x: x[1], reverse=True))
    # removing user itself
    sort = sort[1:]
    return list(map(lambda x: x[0], sort[:number]))

user_user_similar = top_n_idx_sparse(1)
user_user_similar

[325,
 634,
 341,
 310,
 207,
 35,
 195,
 485,
 130,
 229,
 102,
 403,
 119,
 387,
 539,
 575,
 391,
 468,
 497,
 510]

In [25]:
def user_top_movies(user_id, number=10):
    user_rate = rate_df[rate_df['userId'] == user_id]
    sort = user_rate.sort_values(by='rating', ascending=False)
    number = number if number <= len(sort) else len(sort)
    return sort['movieId'].values[:number]

user_325_top_movies = user_top_movies(325)
user_325_top_movies

array([  31, 4085, 1569,  585, 1371, 1061, 1129, 3039, 1339, 1172],
      dtype=int64)

In [26]:
def recommendation_for_user(user_id, number=20):
    similar_users = top_n_idx_sparse(user_id)
    movies = []
    for i in similar_users:
        similar_user_movies = user_top_movies(i)
        [movies.append(j) for j in similar_user_movies]
    temp = rate_df[rate_df['userId'] == user_id]
    for i in movies:
        if len(temp[temp['movieId'] == i]) > 0:
            movies.remove(i)
    titles = [movie_df[movie_df['id'] == str(id)]['title'].values[0] for id in movies if len(movie_df[movie_df['id'] == str(id)]['title']) > 0]
    number = number if number < len(titles) else len(titles)
    return titles[:number]
    
recommendation_for_user(1) 

['Monsters, Inc.',
 'The Million Dollar Hotel',
 'Sleepless in Seattle',
 'Rocky III',
 'Minority Report',
 'Phar Lap',
 'André Hazes, Zij Gelooft in Mij',
 'Two-Minute Warning',
 'Return of the Jedi',
 'American Beauty',
 'Torrente 3 The Protector',
 'Antoine and Colette',
 'Zatoichi',
 'Interview with the Vampire',
 'Blood Diamond',
 'Walk the Line',
 'Terminator 3: Rise of the Machines',
 'Men in Black II',
 'Meet the Parents',
 'Frankenstein Unbound']

# Ensemble

In [48]:
def ensemble_recommendation_intersection_based(user_id, number=10):
    collaborative = recommendation_for_user(user_id)
    content_based = recommen_by_user(user_id)
    result = list(set(collaborative) & set(content_based)) # finding intersect
    for i in result:
        collaborative.remove(i)
        content_based.remove(i)
    collaborative_index = 0
    content_base_index = 0
    while len(result) < number:
        if collaborative_index > content_base_index:
            result.append(content_based[content_base_index])
            content_base_index = content_base_index + 1
        else:
            result.append(collaborative[collaborative_index])
            collaborative_index = collaborative_index + 1
    return result

def ensemble_recommendation_collaborative_based(user_id, number=10):
    collaborative = recommendation_for_user(user_id)
    results = []
    for movie in collaborative:
        recommended_movies = recommend_by_movie(movie)
        for i in recommended_movies:
            results.append(i) if i not in results else None
    return results[:number]



def ensemble_recommendation(user_id, number=10, intersection_base=True):
    if intersection_base:
        return ensemble_recommendation_intersection_based(user_id, number)
    else:
        return ensemble_recommendation_collaborative_based(user_id, number=10)

In [49]:
recommendation = ensemble_recommendation(1,intersection_base=True)
recommendation

['Rocky III',
 'Monsters, Inc.',
 'American Pie',
 'The Million Dollar Hotel',
 'American Reunion',
 'Sleepless in Seattle',
 'Date and Switch',
 'Minority Report',
 'High School Musical 3: Senior Year',
 'Phar Lap']

In [50]:
recommendation = ensemble_recommendation(1, intersection_base=False)
recommendation

['Monsters, Inc.',
 'Monsters University',
 'Girl vs. Monster',
 'Papa ou maman',
 "Mike's New Car",
 '捉妖记',
 'Courageous',
 'Séptimo',
 'Indivisibili',
 'Monster House']

# MLOps